In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

# **Download model**

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HuggingFaceTB/SmolLM2-135M"  # Thay bằng tên mô hình bạn muốn tải

# Tải mô hình và tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Lưu mô hình và tokenizer vào thư mục cục bộ
model.save_pretrained("./smollm2_model_1")
tokenizer.save_pretrained("./smollm2_model_1")

('./smollm2_model_1/tokenizer_config.json',
 './smollm2_model_1/special_tokens_map.json',
 './smollm2_model_1/vocab.json',
 './smollm2_model_1/merges.txt',
 './smollm2_model_1/added_tokens.json',
 './smollm2_model_1/tokenizer.json')

# **Download dataset**

In [1]:
from datasets import load_dataset

# Tải dataset từ Hugging Face
dataset = load_dataset("dchatca/vn_economic_QA")

# Lưu dataset vào thư mục cục bộ
dataset.save_to_disk("./dataset2")


/home/khoi/miniconda3/envs/py1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Saving the dataset (1/1 shards): 100%|██████████| 152/152 [00:00<00:00, 15421.35 examples/s]


In [2]:
from datasets import load_from_disk

# Nạp dataset từ thư mục cục bộ
local_dataset = load_from_disk("./dataset2")

# Kiểm tra dataset
print(local_dataset)


DatasetDict({
    train: Dataset({
        features: ['Question ', 'Answer', 'QA'],
        num_rows: 605
    })
    test: Dataset({
        features: ['Question ', 'Answer', 'QA'],
        num_rows: 152
    })
})


In [8]:
print(dataset["train"][0])  # In ra dòng đầu tiên


{'vi_question': ' lần cuối cùng Richmond chơi trong trận chung kết sơ bộ là khi nào ', 'en_answer': " Richmond Football Club Richmond began 2017 with 5 straight wins, a feat it had not achieved since 1995. A series of close losses hampered the Tigers throughout the middle of the season, including a 5-point loss to the Western Bulldogs, 2-point loss to Fremantle, and a 3-point loss to the Giants. Richmond ended the season strongly with convincing victories over Fremantle and St Kilda in the final two rounds, elevating the club to 3rd on the ladder. Richmond's first final of the season against the Cats at the MCG attracted a record qualifying final crowd of 95,028; the Tigers won by 51 points. Having advanced to the first preliminary finals for the first time since 2001, Richmond defeated Greater Western Sydney by 36 points in front of a crowd of 94,258 to progress to the Grand Final against Adelaide, their first Grand Final appearance since 1982. The attendance was 100,021, the largest 

# **Tokenizer dataset**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_from_disk
from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
import torch
from tqdm import tqdm

# 1. Load model và tokenizer
model_path = "./smollm2_model_1"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

/home/khoi/miniconda3/envs/py1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer
from datasets import load_from_disk, DatasetDict

# 1. Tải model và tokenizer từ local
model_path = "./smollm2_model_1"  # Đường dẫn tới model đã lưu
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 2. Tải dataset từ local
dataset_path = "./dataset"  # Đường dẫn tới dataset đã lưu
dataset = load_from_disk(dataset_path)

# 3. Chọn các cột vi_question và vi_answer
# Map lại để chỉ giữ lại các cột vi_question và vi_answer
selected_columns = dataset.map(
    lambda example: {"input_text": example["vi_question"], "target_text": example["vi_answer"]},
    remove_columns=["vi_question", "en_answer", "vi_answer", "en_question"]
)

# 4. Tokenizer dataset
def tokenize_function(examples):
    # Tokenize các cột input_text và target_text
    return tokenizer(
        examples["input_text"],  # Thay "input_text" bằng tên trường dữ liệu của bạn
        truncation=True,         # Bật truncation
        padding=True,            # Bật padding
        max_length=512           # Đặt độ dài tối đa
    )

# Áp dụng tokenizer lên dataset
tokenized_dataset = selected_columns.map(tokenize_function, batched=True)

# 5. Lưu dataset đã xử lý vào local
output_path = "./tokenized_dataset"  # Đường dẫn lưu dataset đã tokenized
tokenized_dataset.save_to_disk(output_path)

print(f"Dataset đã được lưu tại: {output_path}")

Map:   0%|          | 0/99698 [00:00<?, ? examples/s]


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [11]:
# Nạp dataset từ thư mục cục bộ
dataset1 = load_from_disk("./tokenized_dataset")

# Kiểm tra dataset
print(dataset1)


DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 99698
    })
})


In [12]:
print(dataset1["train"][0]) 

{'input_text': ' lần cuối cùng Richmond chơi trong trận chung kết sơ bộ là khi nào ', 'target_text': ' Câu lạc bộ bóng đá Richmond Richmond bắt đầu năm 2017 với 5 trận thắng liên tiếp, một thành tích mà đội chưa đạt được kể từ năm 1995. Một loạt trận thua sát nút đã cản trở Những chú hổ trong suốt giữa mùa giải, bao gồm trận thua 5 điểm trước Western Bulldogs, thua Fremantle 2 điểm và thua 3 điểm trước Giants. Richmond kết thúc mùa giải một cách mạnh mẽ với những chiến thắng thuyết phục trước Fremantle và St Kilda ở hai vòng đấu cuối cùng, nâng câu lạc bộ lên vị trí thứ 3 trên bảng xếp hạng. Trận chung kết đầu tiên trong mùa giải của Richmond gặp Cats tại MCG đã thu hút một lượng khán giả cuối cùng đủ điều kiện kỷ lục là 95.028; Những chú hổ giành được 51 điểm. Lần đầu tiên tiến đến trận chung kết sơ bộ đầu tiên kể từ năm 2001, Richmond đã đánh bại Greater Western Sydney 36 điểm trước đám đông 94.258 người để tiến tới trận Chung kết gặp Adelaide, trận Chung kết đầu tiên của họ kể từ nă